# Taxi-v3 with Reinforce Learning

In [1]:
import gym

## Initialize environment

In [2]:
env = gym.make("Taxi-v3").env

## Render environment

In [3]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Action space and state space
* States (Taxi row, Taxi col, passenger_location, destination)

In [5]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

state = env.encode(3,1,2,0)
print("State:",state)

env.s = state
env.render()

state = env.encode(3,1,3,2)
print("State:",state)

env.s = state
env.render()

Action Space Discrete(6)
State Space Discrete(500)
State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

State: 334
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Reward table P

In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

# Brute Force algorithm

In [8]:
env.s = 328 # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done :
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame':env.render(mode='ansi'),
        'state':state,
        'action':action,
        'reward':reward
    })
    
    epochs += 1

print("Timesteps taken : {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken : 2592
Penalties incurred: 828


In [10]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, s = .01):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep : {i+1}")
        print(f"State : {frame['state']}")
        print(f"Action : {frame['action']}")
        print(f"Reward : {frame['reward']}")
        sleep(s)

In [11]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep : 2592
State : 0
Action : 5
Reward : 20


# Q-learning
* Q-table size : (# of states, # of actions)

In [14]:
import numpy as np
import random
q_table = np.zeros([env.observation_space.n, env.action_space.n])

## Q-learning Training

In [15]:
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# for plotting metrics
all_epochs = []
all_penalties = []

for i in range(1,100001):
    state = env.reset()
    
    epochs, penalties, reward = 0,0,0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else :
            action = np.argmax(q_table[state]) # Exploit leaned values
        
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state,action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha)*old_value + alpha * (reward + gamma*next_max)
        q_table[state,action] = new_value
        
        if reward == -1:
            penalties += 1
        
        state = next_state
        epochs += 1
    
    if i%100 == 0:
        clear_output(wait=True)
        print(f"Episode : {i}")

Episode : 100000


## Q-learning Testing

In [16]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1
        
        epochs += 1
    
    total_penalties += penalties
    total_epochs += epochs
    
print(f"Results after {episodes} episodes : ")
print(f"Average timesteps per episode : {total_epochs/episodes}")
print(f"Average penalties per episode : {total_penalties/episodes}")

Results after 100 episodes : 
Average timesteps per episode : 13.09
Average penalties per episode : 0.0


## Q-learning : Test(print result)

In [19]:
frames = []

state = env.reset()
epochs, penalties, reward = 0,0,0

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    epochs += 1
    frames.append({
        'frame':env.render(mode='ansi'),
        'state':state,
        'action':action,
        'reward':reward
    })
    
total_penalties += penalties
total_epochs += epochs

In [23]:
print_frames(frames, s=0.3)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep : 12
State : 85
Action : 5
Reward : 20
